Prep FIA data for calibration analysis. 
* Create input database of stands/trees that meet our criteria. 
* Create a groundtruth dataset containing Accretion and Mortality values for FIA data summed to stand level. 

In [141]:
import sqlite3
import pandasql as ps

import pandas as pd
import numpy as np

import openpyxl

import warnings
warnings.filterwarnings('ignore')

In [142]:
OUT_PATH = 'D:/fvs_calibration/data/interim'

In [143]:
#read in FIA source tables 
DB_PATH = 'D:FIA/FIADB_CA.db'

conn = sqlite3.connect(DB_PATH)
# grab fvs_standinit_cond, cond, tree tables, and tree remeasurement table
stands = pd.read_sql_query("select STAND_CN, STAND_ID, INV_YEAR AS INVYR, INV_MONTH, DG_MEASURE from FVS_STANDINIT_COND where VARIANT = 'NC'", conn)
trees = pd.read_sql_query("select STAND_CN, STAND_ID, TREE_CN from FVS_TREEINIT_COND", conn)
cond = pd.read_sql_query("select CN AS STAND_CN, PLT_CN, CONDID, DSTRBCD1, TRTCD1, CONDPROP_UNADJ, COND_STATUS_CD from COND", conn)


#### 1. Create an input FVS database of stands/trees that fit our criteria. 

Filter FVS_Standinit_cond table to only include stands that:
* Have at at least two measurement years
* Have not experienced disturbance between measurements
* Forested stand
* Ten years between measurements - (grow season apr 1 - sept 30)

In [144]:
# some pre-processing
# create STAND_ID2 - scrape stand_id so remeasurement ids match
stands['STAND_ID2'] = stands['STAND_ID'].str[0:3] + stands['STAND_ID'].str[10:]

In [145]:
# identify stands that have had no disturbance or harvest
stands_nodisturb = ps.sqldf("""select s.STAND_CN, s.STAND_ID, s.STAND_ID2, s.INVYR, s.INV_MONTH, s.DG_MEASURE, c.PLT_CN, c.CONDID, c.CONDPROP_UNADJ
                               from stands s 
                               join cond c 
                               on s.STAND_CN = c.STAND_CN 
                               WHERE (c.DSTRBCD1 = NULL OR c.DSTRBCD1 = 0)
                               AND (c.TRTCD1 = NULL OR c.TRTCD1 = 0) 
                               AND (c.COND_STATUS_CD = 1)
                          """)
stands_nodisturb.shape[0]

1836

In [146]:
# further filter out stands that don't have remeasurement data 
stands = stands_nodisturb.sort_values(by=['STAND_ID2', 'INVYR'])
stands_filtered = stands[stands.duplicated(subset=['STAND_ID2'], keep=False)]
stands_filtered = stands_filtered[['STAND_CN', 'STAND_ID', 'INVYR', 'INV_MONTH', 'DG_MEASURE', 'STAND_ID2', 'PLT_CN', 'CONDID']]
stands_filtered.shape[0]

1202

In [147]:
# determine years between measurement based on number of grow seasons. 
# each month in a grow month is assigned a value based on growth month
stands = stands_filtered
conditions = [
    (stands['INV_MONTH'] == 5),
    (stands['INV_MONTH'] == 6),
    (stands['INV_MONTH'] == 7),
    (stands['INV_MONTH'] == 8),
    (stands['INV_MONTH'] == 9)
]
values = [2, 4, 6, 8, 10]
stands['month_grow'] = np.select(conditions, values, default=0)

# split dataframe by 1st and 2nd measurement
first = stands.drop_duplicates(subset = ['STAND_ID2'], keep = 'first')[['STAND_ID2', 'INVYR', 'month_grow', 'DG_MEASURE']]\
    .rename(columns={'INVYR': 'invyr1', 'month_grow': 'month1', 'DG_MEASURE':'dg1'})
second = stands.drop_duplicates(subset = ['STAND_ID2'], keep = 'last')[['STAND_ID2', 'INVYR', 'month_grow', 'DG_MEASURE']]\
    .rename(columns={'INVYR': 'invyr2', 'month_grow': 'month2', 'DG_MEASURE':'dg2'})
# recombine in wide format
mix = first.merge(second, how='inner', on='STAND_ID2')
# growth/year calcuation
mix['calc_years'] = mix['dg2'] + ((mix['month2']-mix['month1'])/10)

mix = mix[['STAND_ID2', 'calc_years']]

stands_filtered2 = stands_filtered.merge(mix, how='inner', on='STAND_ID2')[['STAND_CN', 'STAND_ID', 'INVYR', 'STAND_ID2', 'PLT_CN', 'CONDID', 'calc_years']]

print(stands_filtered2.shape[0])
stands_filtered2.head(1)

1202


,STAND_CN,STAND_ID,INVYR,STAND_ID2,PLT_CN,CONDID,calc_years
0,23652219010900,00062001050101015506791,2001.0,0000101015506791,23652215010900,1,10.4


In [148]:
# print second measurement years for later
print(first['invyr1'].unique())
print(second['invyr2'].unique())

[2001. 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010.]
[2011. 2012. 2013. 2014. 2015. 2017. 2016. 2018. 2019.]


In [149]:
# select first measurment year for our filtered years for input database
stand_grow_id = stands_filtered2[['STAND_ID', 'calc_years']]
stands_init = stands_filtered2.drop_duplicates(subset = ['STAND_ID2'], keep = 'first')
stands_init = stands_init[['STAND_ID']]
stands_init.shape[0]

601

Create an input database from filtered stands

In [150]:
# read tables in from CA FIA database
fvs_stands = pd.read_sql_query("select * from FVS_STANDINIT_COND where VARIANT = 'NC'", conn)
fvs_trees = pd.read_sql_query("select * from FVS_TREEINIT_COND", conn)

In [151]:
fvs_stands_init = fvs_stands.merge(stands_init, how='inner', on='STAND_ID')
fvs_stands_init = fvs_stands_init[['STAND_CN', 'STAND_ID', 'VARIANT', 'INV_YEAR', 'GROUPS', 'ADDFILES', 'LATITUDE', 'LONGITUDE', 'REGION', 'FOREST', 'DISTRICT', 'LOCATION', 'ECOREGION',
    'AGE', 'ASPECT', 'SLOPE', 'ELEVATION', 'ELEVFT', 'BASAL_AREA_FACTOR', 'INV_PLOT_SIZE', 'BRK_DBH', 'NUM_PLOTS', 'SAM_WT', 'DG_TRANS', 'DG_MEASURE', 'HTG_TRANS', 'HTG_MEASURE',
    'SITE_SPECIES', 'SITE_INDEX', 'SITE_INDEX_BASE_AG', 'PHYSIO_REGION', 'FOREST_TYPE_FIA', 'STATE', 'COUNTY', 'FUEL_MODEL']]
fvs_stands_init.shape[0]

601

Tree List
* Filter FVS_Treeinit_cond table to filtered stand list:

In [152]:
# create FVS_treeinit table containing trees for our filtered stands.
fvs_trees_init = fvs_trees.merge(stands_init, how = 'inner', on = 'STAND_ID')

fvs_trees_init.head(1)

,STAND_CN,STAND_ID,PLOT_CN,STANDPLOT_CN,STANDPLOT_ID,PLOT_ID,TREE_CN,TREE_ID,TAG_ID,AZIMUTH,...,PV_REF_CODE,TOPOCODE,SITEPREP,CREATED_BY,CREATED_DATE,CREATED_IN_INSTANCE,MODIFIED_BY,MODIFIED_DATE,MODIFIED_IN_INSTANCE,VERSION
0,446767010497,00062008050801045666311,29394784010497,446767010497_1,00062008050801045666311_1,1.0,721791010497,500.0,None,51.0,...,None,None,None,JDSHAW,2020-08-13 12:18:55,290487,None,None,None,2.0.12


In [153]:
# create and send tables to an input FVS sqlite database
conn2 = sqlite3.connect('../data/FVSIn.db')
fvs_stands_init.to_sql('fvs_standinit', conn2, if_exists='replace', index=False)
fvs_trees_init.to_sql('fvs_treeinit', conn2, if_exists = 'replace', index=False)

24555

#### 2. Create list of filtered FIA trees and calculate TPA

* Filter FIA tree records to only include trees that are in our subset of stands
* Filter out trees from Time2 that are not present at Time1
* Calculate TPA to match FVS TPA calculations

Read in FIA tree data with volume calculated in NVEL

In [154]:
volume_calc = pd.read_excel('../data/NC_trees_for_NVEL_calc.xlsx' )

Filter FIA tree records to only include trees that are in our subset of stands

In [155]:
# add key values to volume_calc
volume = volume_calc
volume['STAND_CN'] = volume['STAND_CN'].astype(str)
volume['TRE_CN'] = volume['TRE_CN'].astype(str)
stands_filtered_id = stands_filtered2[['STAND_CN', 'STAND_ID']]
volume = volume.merge(stands_filtered_id, on = 'STAND_CN')

# add cond_prop for later calculations
cond_prop = cond[['STAND_CN', 'CONDPROP_UNADJ']]
volume = volume.merge(cond_prop, on='STAND_CN')

volume = volume[['STAND_CN', 'STAND_ID', 'INVYR', 'TRE_CN', 'TREE', 'STATUSCD', 'TPA_UNADJ', 'CONDPROP_UNADJ', 'VOLUME_CVTS', 'VOLUME_CV4']]
print(volume.shape[0])
volume.head(1)

46137


,STAND_CN,STAND_ID,INVYR,TRE_CN,TREE,STATUSCD,TPA_UNADJ,CONDPROP_UNADJ,VOLUME_CVTS,VOLUME_CV4
0,23652219010900,00062001050101015506791,2001,23652246010900,100,1,6.018046,1.0,6.742738,2.9


Remove stands from FVS Input database that don't have trees associated with them in the volume database

In [156]:
# check that all stands in our stand list have trees
list = volume['STAND_ID'].unique().tolist()
fvs_stands_init2 = fvs_stands_init[fvs_stands_init['STAND_ID'].isin(list)]
fvs_stands_init2.to_sql('fvs_standinit', conn2, if_exists='replace', index=False)

527

Calculate TPA to match FVS TPA calculations

In [157]:
volume['TPA'] = volume['TPA_UNADJ']/volume['CONDPROP_UNADJ']

Filter out trees from Time 2 that are not present at Time 1

In [158]:
# read in Prev_tre_cn value from FIA TREE table
fia_trees = pd.read_sql_query("select CN AS TRE_CN, PREV_TRE_CN from TREE", conn)

In [159]:
# join with the volume table on TRE_CN
filter = volume.merge(fia_trees, on='TRE_CN', how='inner')
filter.head(1)

# split dataframe by 1st and 2nd measurement
first = filter.loc[filter['INVYR'] < 2011][['TRE_CN']]
second = filter.loc[filter['INVYR'] > 2010][['TRE_CN', 'PREV_TRE_CN']].rename(columns={'TRE_CN': 'TRE_CN2'})

# filter second record to trees only present in time 1
second_filter = second.merge(first, how = 'inner', left_on='PREV_TRE_CN', right_on='TRE_CN')[['TRE_CN2']].rename(columns={'TRE_CN2': 'TRE_CN'})

# concatenate list of Tre_cns
tree_ids = pd.concat([first, second_filter])
# filter volume tree records to only include those trees
volume = volume.merge(tree_ids, how='inner', on='TRE_CN')

#### 3. Calculate groundtruth values from filtered FIA tree data

Metric calculation descriptions
* **Accretion**: Summed tree tcuft values as calcuated by NVEL expanded using manually calculated TPA values from FIA (tpa_unadj/condprop) + Mort Value
* **Mortality**: Summed tree tcuft values as calcuated by NVEL, for dead trees, expanded using manually calculated TPA values from FIA (tpa_unadj/condprop)

Prep data
* Expand volumes by TPA
* Update STATUSCD values

In [161]:
#expand by TPA
volume['vol_exp'] = volume['VOLUME_CVTS'] * volume['TPA']

#get common on the tree value/STATUSCD values
volume.loc[(volume['STATUSCD'] == 6), 'STATUSCD'] = 7
volume.loc[(volume['STATUSCD'] == 8), 'STATUSCD'] = 9

In [162]:
#sum by stand
volume_agg = volume.groupby(['STAND_ID', 'INVYR', 'STATUSCD'], as_index=False).agg({'vol_exp': 'sum'})
volume_agg = volume_agg.merge(stand_grow_id, how = 'left', on = 'STAND_ID')

In [163]:
# need find difference between remeasurement years and normalize by calc_years

# create STAND_ID2 - scrape stand_id so remeasurement ids match
volume_agg['STAND_ID2'] = volume_agg['STAND_ID'].str[0:3] + volume_agg['STAND_ID'].str[10:]

# split dataframe by 1st and 2nd measurement
first = volume_agg.loc[volume_agg['INVYR'] < 2011].rename(columns={'STAND_ID': 'STAND_ID_1', 'INVYR':'INVYR_1', 'vol_exp':'vol_exp_1'})
second = volume_agg.loc[volume_agg['INVYR'] > 2010].rename(columns={'STAND_ID': 'STAND_ID_2', 'INVYR':'INVYR_2', 'vol_exp':'vol_exp_2'})

join = first.merge(second, how = 'outer', on = ['STAND_ID2','STATUSCD', 'calc_years'])
join = join.loc[join['STATUSCD'] != 9]
join['vol_exp_1'] = join['vol_exp_1'].fillna(0)

# calculate differenece in volume for mortality and accretion
join['diff'] = join['vol_exp_2'] - join['vol_exp_1']

In [164]:
# calculate annual growth/mortality value, divide by decimal growth years
join['diff_annual'] = join['diff']/join['calc_years']

In [175]:
# clean it all up
# create mort df
mort = join.loc[join['STATUSCD'] == 7][['STAND_ID2', 'diff_annual']].rename(columns={'diff_annual':'mort'})
# create acc df
acc = join.loc[join['STATUSCD'] == 1][['STAND_ID2', 'STAND_ID_1', 'diff_annual']].rename(columns={'diff_annual':'growth'})

#join em
values = acc.merge(mort, how='left', on=('STAND_ID2')).dropna(subset='STAND_ID_1').rename(columns={'STAND_ID_1':'STAND_ID'})
values['mort'] = values['mort'].fillna(0)
values['acc'] = values['growth']+values['mort']

In [177]:
# export groundtruth data to csv 
values.to_excel('../data/groundtruth.xlsx' )

In [178]:
conn.close()
conn2.close()